---------------
---------------
# ***------------------------------------- PART 1 / 3 -------------------------------------***
---------------
---------------

In [1]:
import numpy as np
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import requests
import geocoder
import folium

Let's retrieve the data from Wikipedia and import them into a dataframe with read_html:

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
toronto_raw = pd.read_html(url)

What is the type of this object ?

In [3]:
print(type(toronto_raw))

<class 'list'>


What is its length ?

In [4]:
print(len(toronto_raw))

3


Let's examine each element and try to find if one of the may correspond to our need...

In [5]:
first, second, third = toronto_raw
first

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


second and third appeared to be much less interesting than first, so let's use this one:

In [6]:
toronto = first

And let's del rows where 'Borough' = 'Not assigned'

In [7]:
toronto = toronto[toronto['Borough'] != "Not assigned"]

In [8]:
toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Now reset the indexes:

In [9]:
toronto.reset_index(drop=True, inplace=True)
toronto.head(10)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Eventually, let's look at its shape :

In [10]:
toronto.shape

(103, 3)

---------------
---------------
# ***------------------------------------- PART 2 / 3 -------------------------------------***
---------------
---------------

In [11]:
# !pip install geocoder

Note : as Google API did not return any result, I used arcgis instead.

In [12]:
lat_list = []
long_list = []
i = 1
for code in toronto['Postal Code']:
    g = geocoder.arcgis(f'{code}, Toronto, Ontario')
    lat_list.append(g.json['lat'])
    long_list.append(g.json['lng'])
    if i%10 == 0: print(f'{i}...', end = ' ', flush=True)
    i += 1
print('Done!')   

10... 20... 30... 40... 50... 60... 70... 80... 90... 100... Done!


In [13]:
toronto.loc[:,'Latitude'] = lat_list
toronto.loc[:,'Longitude'] = long_list
toronto.head()

c:\users\sophi\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\sophi\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


Coordinates seem to be slightly different... (e.g : M5A Downton Toronto is 43.65512, -79.36264 instead of 43.654250, -79.360636 which represent about 190m).
Maybe because I did not use the Google API ?
To avoid any problem I would rather use the csv file data but it seems not to be available (Error 501).

---------------
---------------
# ***------------------------------------- PART 3 / 3 -------------------------------------***
---------------
---------------

Let's select only Boroughs that contain the word "Toronto":

In [14]:
toronto = toronto[toronto['Borough'].str.contains('Toronto')]
toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
15,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
19,M4E,East Toronto,The Beaches,43.67709,-79.29547


Now let's retrieve information from Foursquare :

In [15]:
CLIENT_ID = '...' # your Foursquare ID
CLIENT_SECRET = '...' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [17]:
def get_nearby_venues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
           
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = get_nearby_venues(names=toronto['Neighbourhood'],
                                  latitudes=toronto['Latitude'], 
                                  longitudes=toronto['Longitude'])
toronto_venues.head()

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's transform our dataframe so that we have numerical values instead of strings:

In [19]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.0,0.00,0.016667,0.0,0.016667,0.000000,0.000000,0.000000,...,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.016667
1,"Brockton, Parkdale Village, Exhibition Place",0.011494,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.022989,0.011494,...,0.000000,0.011494,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.011494
2,"Business reply mail Processing Centre, South C...",0.000000,0.0,0.02,0.000000,0.0,0.010000,0.000000,0.010000,0.030000,...,0.000000,0.020000,0.000000,0.000000,0.000000,0.010000,0.0,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.013158,...,0.013158,0.000000,0.013158,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.013158
4,Central Bay Street,0.000000,0.0,0.00,0.000000,0.0,0.013158,0.013158,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.013158,0.013158,0.013158,0.0,0.0,0.0,0.000000


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Restaurant,Bakery,Farmers Market,Cocktail Bar,Beer Bar,Cheese Shop,Breakfast Spot,Seafood Restaurant,Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bar,Coffee Shop,Restaurant,Sandwich Place,Gift Shop,Nightclub,Supermarket,Japanese Restaurant,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Restaurant,Italian Restaurant,Café,Asian Restaurant,Bar,Sushi Restaurant,Seafood Restaurant,Pub
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Bar,Park,French Restaurant,Restaurant,Speakeasy,Sandwich Place,Bakery
4,Central Bay Street,Coffee Shop,Clothing Store,Plaza,Bookstore,Hotel,Middle Eastern Restaurant,Sandwich Place,Café,Restaurant,Sushi Restaurant


It's time for clustering ! 

In [22]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_ 
labels

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 4, 4, 1, 4, 4, 3, 4, 1, 1,
       4, 4, 4, 4, 1, 4, 4, 4, 4, 4, 0, 4, 1, 4, 4, 4])

Let's add labels to our dataset...

In [23]:
neighborhoods_venues_sorted.insert(0, 'Cluster', labels)
toronto_merged = toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.dropna(inplace=True)
toronto_merged['Cluster']= toronto_merged['Cluster'].astype('int')

And eventually, let's create a map that shows colored clusters.

In [24]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'],toronto_merged['Neighbourhood'], toronto_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters